In [1]:
using Oceananigans

In [2]:
using TopographicHorizontalConvection: HorizontalConvectionSimulation

In [4]:
simulation = HorizontalConvectionSimulation(Ra=1e5, h₀_frac=0.6, Nx= 256, Ny=1, Nz=32, advection=true)

Simulation of NonhydrostaticModel{CPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 17.678 ms
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 5.267 minutes
├── Stop iteration : Inf
├── Wall time limit: Inf
├── Callbacks: OrderedDict with 6 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   ├── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
│   ├── wizard => Callback of TimeStepWizard(cfl=0.5, max_Δt=Inf, min_Δt=0.0) on IterationInterval(50)
│   └── progress => Callback of (::TopographicHorizontalConvection.var"#progress#9") on IterationInterval(10)
├── Output writers: OrderedDict with 6 entries:
│   ├── checkpointer => Checkpointer{TimeInterval, Vector{Symbol}}


In [5]:
run!(simulation, pickup=false)

┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/Simulations/run.jl:184


i:      0, sim time:  0.000, wall time:  0 seconds, Δt:  0.0194, advective CFL: 0.00e+00, diffusive CFL: 6.30e-02


┌ Info:     ... simulation initialization complete (46.273 seconds)
└ @ Oceananigans.Simulations /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/Simulations/run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/Simulations/run.jl:117
┌ Info:     ... initial time step complete (22.942 seconds).
└ @ Oceananigans.Simulations /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/Simulations/run.jl:126


i:     10, sim time:  0.194, wall time: 1.178 minutes, Δt:  0.0194, advective CFL: 2.96e-04, diffusive CFL: 6.30e-02
i:     20, sim time:  0.389, wall time: 1.196 minutes, Δt:  0.0194, advective CFL: 9.67e-04, diffusive CFL: 6.30e-02
i:     30, sim time:  0.583, wall time: 1.218 minutes, Δt:  0.0194, advective CFL: 1.86e-03, diffusive CFL: 6.30e-02
i:     40, sim time:  0.778, wall time: 1.238 minutes, Δt:  0.0194, advective CFL: 2.90e-03, diffusive CFL: 6.30e-02
i:     50, sim time:  0.972, wall time: 1.256 minutes, Δt:  0.0214, advective CFL: 4.49e-03, diffusive CFL: 6.93e-02
i:     60, sim time:  1.171, wall time: 1.280 minutes, Δt:  0.0214, advective CFL: 5.94e-03, diffusive CFL: 6.93e-02
i:     70, sim time:  1.385, wall time: 1.305 minutes, Δt:  0.0214, advective CFL: 7.62e-03, diffusive CFL: 6.93e-02
i:     80, sim time:  1.599, wall time: 1.325 minutes, Δt:  0.0214, advective CFL: 9.42e-03, diffusive CFL: 6.93e-02
i:     90, sim time:  1.813, wall time: 1.344 minutes, Δt:  0.02

┌ Info: Simulation is stopping after running for 22.593 minutes.
└ @ Oceananigans.Simulations /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/Simulations/simulation.jl:202
┌ Info: Simulation time 5.267 minutes equals or exceeds stop time 5.267 minutes.
└ @ Oceananigans.Simulations /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/Simulations/simulation.jl:203


In [5]:
using NCDatasets
using CairoMakie
using Printf
using Oceananigans.Fields
using Oceananigans.AbstractOperations: volume
using NaNStatistics

In [7]:
@info "Creating animation from saved data ..."
ds = NCDataset(string("/Users/hfdrake/code/HorizontalConvection/output/turbulent_h0.6_Ra1.0e8_HydPresAnom_buoyancy.nc"));

times = ds["time"][:]

n = Observable(1)

x = ds["xC"][4+1:end-4]
z = ds["zC"][4+1:end-4]

H =1 
Lx = 8H

bₙ = @lift ds["b"][4+1:end-4, 1, 4+1:end-4, $n]

fig = Figure(size = (800, 300))

kwargs = (xlabel=L"x/H", 
               ylabel=L"z/H", 
               limits = ((-Lx/2, Lx/2), (-H, 0)), 
               aspect = Lx/H, 
               titlesize = 20)

title = @lift @sprintf("nondimensional buoyancy at t = %.2f", times[$n])

ax_B = Axis(fig[1,1]; title = title, kwargs...)

B_lims = (-0.6, 0.6)

b = ds["b"][4+1:end-4, 1, 4+1:end-4, 3] 
wet = b.!=0.
b[.!wet] .= NaN
wet_masked = Float64.(copy(wet))
wet_masked[wet] .= NaN

hm_b = heatmap!(ax_B, x, z, bₙ; colorrange = B_lims, colormap = :balance)
hm = heatmap!(ax_B, x, z, wet_masked[:,:], colormap = :algae)
Colorbar(fig[1,2], hm_b)

frames = 1:length(times)

record(fig, "../animations/HCRa1e8HydPresAnom.mp4", frames, framerate = 8) do i 
    n[] = i 
end

close(ds)

┌ Info: Creating animation from saved data ...
└ @ Main /Users/hfdrake/code/HorizontalConvection/notebooks/VariableRayleighNum.ipynb:1


closed Dataset